In [ ]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from prettytable import PrettyTable
import seaborn as sns

from typing import List, Dict, Tuple

METRICS = ["roc_auc", "f1", "precision", "recall", "specificity", "jw5", "jw6"]

In [ ]:
load_dotenv()
project_path = os.getenv("PROJECTPATH")
data_path = os.getenv("DATAPATH")

experiment_name = "ct_rate_cardiomegaly_patch"
label = "Cardiomegaly"
results_path = os.path.join(project_path, "runs", experiment_name, "results", label)

In [ ]:
def get_logits(result_path: str, valid_name: int, epoch: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Folds and epochs are 0-indexed.
    """
    epoch_predictions_path = os.path.join(
        result_path, valid_name, "predictions", f"epoch_{epoch:02d}.csv"
    )
    epoch_predictions = pd.read_csv(epoch_predictions_path)
    logits = epoch_predictions["logits"].values
    labels = epoch_predictions["labels"].values

    return logits, labels

In [ ]:
def get_metrics(logits: np.ndarray, labels: np.ndarray) -> Dict[str, float]:
    """
    Calculate performance metrics based on logits and true labels.
    """
    roc_auc = roc_auc_score(labels, logits)

    probabilities = 1 / (1 + np.exp(-logits))
    thresholds = np.linspace(0, 1, 100)
    best_threshold = 0.5
    best_youden = 0

    for threshold in thresholds:
        predictions = (probabilities >= threshold).astype(int)

        sensitivity = np.sum((predictions == 1) & (labels == 1)) / np.sum(labels == 1)
        specificity = np.sum((predictions == 0) & (labels == 0)) / np.sum(labels == 0)
        youden = sensitivity + specificity - 1

        if youden > best_youden:
            best_youden = youden
            best_threshold = threshold

    final_predictions = (probabilities >= best_threshold).astype(int)

    final_precision = np.sum((final_predictions == 1) & (labels == 1)) / np.sum(final_predictions == 1)
    final_recall = np.sum((final_predictions == 1) & (labels == 1)) / np.sum(labels == 1)
    specificity = np.sum((final_predictions == 0) & (labels == 0)) / np.sum(labels == 0)
    jw5 = (final_precision + final_recall) / 2
    jw6 = final_precision*0.4 + final_recall*0.6
    best_f1 = 2 * final_precision * final_recall / (final_precision + final_recall)

    return {
        "roc_auc": roc_auc, 
        "f1": best_f1,
        "precision": final_precision,
        "recall": final_recall,
        "specificity": specificity,
        "jw5": jw5,
        "jw6": jw6,
    }

In [ ]:
def get_cv_results(
        results_path: str,
        n_folds: int = 5,
        n_epochs: int = 10
    ) -> Tuple[Dict[str, List[float]], Dict[str, List[Tuple[float, float]]]]:
    """
    Parse cross-validation results. Returns metrics for each epoch and fold, along with confidence intervals.

    Args:
        results_path: Path to results directory.
        n_folds: Number of folds.
        n_epochs: Number of epochs.

    Returns:
        metrics_epoch_fold: Dictionary of metrics for each epoch and fold. 
        confidence_intervals: Dictionary of mean and standard deviation for each metric and each epoch.
    """
    epochs_performance = {m: {ep: [] for ep in range(n_epochs)} for m in METRICS}

    for i in range(n_folds):
        for ep in range(n_epochs):
            logits, labels = get_logits(results_path, f"fold_{i}", ep)

            performance_metrics = get_metrics(logits, labels)
            for metric_name, metric_value in performance_metrics.items():
                epochs_performance[metric_name][ep].append(metric_value)

    confidence_intervals = {
        m: [(np.mean(perf), np.std(perf)) for perf in epochs_performance[m].values()]
        for m in METRICS
    }

    return epochs_performance, confidence_intervals

In [ ]:
def best_epoch_from_metric(
        epochs_performance: Dict[str, List[float]],
        selection_metric: str = "roc_auc"
    ) -> int:
    """
    Select the best epoch based on the specified metric.
    """
    epoch_mean_selection = [
        np.mean(perf) for perf in epochs_performance[selection_metric].values()
    ]

    best_epoch = np.argmax(epoch_mean_selection)

    return int(best_epoch)

In [ ]:
def generate_simple_folds_plot(
        epochs_performance: Dict[str, List[float]],
        label:str,
        metric:str,
        num_folds=5,
        num_epochs=10,
        cm=plt.cm.tab10
    ) -> None:
    plt.figure()

    for fold_idx in range(num_folds):
        epoch_metrics = [epochs_performance[metric][x][fold_idx] for x in range(num_epochs)]
        plt.plot(epoch_metrics, color=cm(fold_idx), label=f"Fold {fold_idx + 1}")

    plt.xlabel("Epoch")
    plt.ylabel(metric)
    plt.legend()
    plt.title(f"Training Curve {metric} for {label}")

    plt.show()

In [ ]:
def generate_ci_plot(
        confidence_intervals:Dict[str, List[Tuple[float, float]]],
        label:str,
        metric:str,
        num_epochs=10,
        z_score=1.96,
        ci_label:str = "95% CI",
        full_y:bool = False
    ) -> None:
    plt.figure()

    means = [ci[0] for ci in confidence_intervals[metric]]
    stds = [ci[1] for ci in confidence_intervals[metric]]
    ci = [std * z_score for std in stds]
    
    epochs = range(num_epochs)
    plt.plot(epochs, means, label=f"Mean {metric}")
    if full_y:
        plt.ylim([0, 1])
    plt.xticks(epochs)
    plt.fill_between(epochs, np.array(means) - np.array(ci), np.array(means) + np.array(ci), alpha=0.2, label=ci_label)
    plt.xlabel("Epoch")
    plt.ylabel(metric)
    plt.title(f"Confidence Interval for {metric} - {label}")
    plt.legend()
    plt.show()
    

In [ ]:
def generate_val_epoch_plot(results_path, valid_name, epoch):

    logits, labels = get_logits(results_path, valid_name, epoch)

    prob_pos = logits[labels==1]
    prob_neg = logits[labels==0]
    
    sns.kdeplot(prob_pos, label="Positive", fill=True, alpha=0.5)
    sns.kdeplot(prob_neg, label="Negative", fill=True, alpha=0.5)

    plt.xlabel("Predicted Probability")
    plt.ylabel("Density")
    plt.title("Density of Predicted Probabilities by Class")
    #plt.xlim((0, 1))
    plt.legend()
    plt.grid(True)

    plt.show()

In [ ]:
generate_val_epoch_plot(results_path, "test", 0)

In [ ]:
epochs_performance, confidence_intervals = get_cv_results(results_path)

In [ ]:
best_epoch_by_metric = {m:best_epoch_from_metric(epochs_performance, selection_metric=m) for m in METRICS} 

table = PrettyTable(["Metric", "Best Epoch"])

for key, value in best_epoch_by_metric.items():
    table.add_row([key, value])
print(table)

In [ ]:
generate_simple_folds_plot(epochs_performance, label, "roc_auc")

In [ ]:
generate_simple_folds_plot(epochs_performance, label, "jw5")

In [ ]:
generate_ci_plot(confidence_intervals, label, "roc_auc")